## 处理数据

In [1]:
from datasets import load_dataset

raw_data_path = "../data/bfcl_multi_turn.json"
raw_ds = load_dataset("json", data_files=raw_data_path, split="train")
# 采样5k
raw_ds = raw_ds.shuffle(seed=42).select(range(5000))

/dfs/data/uv-venv/huggingface/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("../data/system_prompt.txt", "w") as f:
    f.write(raw_ds[0]["0"]["content"].split('[{"name"')[0])

In [3]:
def convert_to_openai(example):
    """
    将 HardGen 格式的单条样本转换为 OpenAI messages 格式
    """
    messages = []

    # --- A. 处理 System Prompt (Key "0") ---
    # dataset 加载后，key 对应的值如果是 None 表示该样本没有此轮（因为是稀疏列）
    if example.get("0") is not None:
        system_data = example["0"]
        system_content = system_data.get("content", "")

        # System 包含 content (及 tools 定义)
        messages.append({"role": "system", "content": system_content})

    # --- B. 处理对话轮次 (Key "1", "2", "3"...) ---
    # 获取所有是数字的 key，排除 "0"
    all_keys = list(example.keys())
    turn_keys = [k for k in all_keys if k.isdigit() and k != "0"]

    # 必须按数字大小排序，保证对话顺序
    sorted_keys = sorted(turn_keys, key=int)

    for key in sorted_keys:
        turn_data = example[key]

        # 因为 dataset 是稀疏的，某些样本可能没有 key "10"，值为 None
        if turn_data is None:
            continue

        role = turn_data.get("role")
        content = turn_data.get("content")

        # --- C. 映射逻辑 ---
        if role == "user":
            messages.append({"role": "user", "content": content})

        elif role == "assistant":
            # 保留 <think> 和 function call
            messages.append({"role": "assistant", "content": content})

        elif role == "tool":
            messages.append({"role": "tool", "content": content})

    # 返回新的格式
    return {"messages": messages}

In [4]:
converted_ds = raw_ds.map(convert_to_openai, remove_columns=raw_ds.column_names)

In [5]:
converted_ds[0]

{'messages': [{'content': 'You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out.\nYou should only return the function calls in your response.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nAt each turn, you should try your best to complete the tasks requested by the user within the current turn. Continue to output functions to call until you have fulfilled the user\'s request to the best of your ability. Once you have no more functions to call, the system will consider the current turn complete and proceed to the next 

## 加载huggingface ecosystem

In [6]:
model_path = "/dfs/data/models/Qwen3-4B-Thinking-2507"

In [7]:
import torch
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)

In [8]:
processor = AutoTokenizer.from_pretrained(model_path)

In [21]:
processor.padding_side

'right'

In [9]:
prompt_demo = processor.apply_chat_template(
    converted_ds[0]["messages"], add_generation_prompt=False, tokenize=False
)

In [10]:
with open("../data/prompt_demo.txt", "w") as f:
    f.write(prompt_demo)

In [11]:
def format_and_mask_last_turn(example):
    """
    处理单条样本：应用模板并只保留最后一次回复的 Loss
    """
    messages = example["messages"]

    # 1. 完整对话 (Prompt + Response)
    # add_generation_prompt=False 因为我们这里是训练，不需要模型生成
    full_text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )

    # 2. 上下文 (Prompt only, 也就是除去最后一条 assistant 的回复)
    # 注意：这里假设最后一条一定是 assistant。如果你的数据不干净，需要加检查。
    if messages[-1]["role"] != "assistant":
        # 如果最后不是 assistant，这整条数据都不应该训练（或者你需要截断）
        # 这里简单处理：返回空，之后过滤掉
        return {"input_ids": [], "labels": [], "attention_mask": []}

    context_messages = messages[:-1]
    context_text = processor.apply_chat_template(
        context_messages, tokenize=False, add_generation_prompt=False
    )

    # 3. Tokenize
    # 加上 return_tensors='pt' 方便后续操作，最后再转回 list
    full_tokens = processor(full_text, return_tensors="pt", add_special_tokens=False)
    context_tokens = processor(
        context_text, return_tensors="pt", add_special_tokens=False
    )

    input_ids = full_tokens["input_ids"][0]
    attention_mask = full_tokens["attention_mask"][0]

    # 4. 构造 Labels 并进行 Masking
    labels = input_ids.clone()

    # 计算上下文的长度，这就是 mask 的分界线
    # 注意：有些 processor 在拼接时可能会有额外的换行符或特殊 token 处理，
    # 这种分别 tokenize 再计算长度的方法通常是最稳健的。
    context_len = context_tokens["input_ids"].shape[1]

    # 将上下文部分的 label 设为 -100 (PyTorch CrossEntropyLoss 默认忽略 -100)
    labels[:context_len] = -100

    return {
        "input_ids": input_ids.tolist(),
        "attention_mask": attention_mask.tolist(),
        "labels": labels.tolist(),
    }

In [12]:
processed_dataset = converted_ds.map(
    format_and_mask_last_turn,
    remove_columns=converted_ds.column_names,  # 移除原始列，只留 input_ids, labels 等
)

In [20]:
# 获取 input_ids 列的所有长度列表
lengths = [len(x) for x in processed_dataset["input_ids"]]

# 打印最大长度
max_len = max(lengths)
print(f"数据集中最长的序列长度是: {max_len}")

数据集中最长的序列长度是: 13246


## 训练

In [13]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=processor,
    padding=True,
    pad_to_multiple_of=8,  # 这是一个好习惯，利用 Tensor Core 加速
)

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 开启 4-bit 加载
    bnb_4bit_quant_type="nf4",  # 使用 NF4 (Normal Float 4) 数据类型，精度更高
    bnb_4bit_compute_dtype=torch.bfloat16,  # 计算时使用的精度 (3090 支持 bf16，推荐)
    bnb_4bit_use_double_quant=True,  # 开启双重量化，进一步节省显存
)

In [15]:
peft_config = LoraConfig(
    r=16,  # LoRA 的秩
    lora_alpha=32,  # Alpha 系数
    lora_dropout=0.05,  # Dropout
    bias="none",  # 偏置项设置
    task_type="CAUSAL_LM",  # 任务类型
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # 需要微调的模块
    use_dora=True,  # <--- 开启 DoRA 模式
)

In [16]:
# 【关键步骤 A】加载模型时传入 quantization_config
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,  # <--- 这里应用量化配置
    device_map="auto",
    # 强烈建议开启 Flash Attention 2
    # attn_implementation="flash_attention_2",
    attn_implementation="sdpa",
)

# 【关键步骤 B】准备模型进行 k-bit 训练
# 这一步会冻结原模型参数，并确保 LayerNorm 保持 FP32 以保证数值稳定性
model = prepare_model_for_kbit_training(model)

# 【关键步骤 C】将 DoRA/LoRA 适配器挂载到模型上
# 普通 Trainer 不接受 peft_config 参数，必须先手动把模型变成 PeftModel
model = get_peft_model(model, peft_config)

# 打印可训练参数，确认只有 LoRA 部分是可训练的
model.print_trainable_parameters()

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.91s/it]


trainable params: 12,109,824 || all params: 4,034,577,920 || trainable%: 0.3002


In [17]:
import os

os.environ["SWANLAB_PROJECT"] = "hardtry"  # 指定项目名

training_args = TrainingArguments(
    output_dir="../checkpoints",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # 显存够的话可以适当增加 accumulating
    num_train_epochs=1,
    learning_rate=1e-4,  # LoRA 通常需要比全量微调稍大的 LR (2e-4 或 1e-4)
    logging_steps=10,
    fp16=False,
    bf16=True,  # 与 bnb_4bit_compute_dtype 保持一致 (3090支持)
    remove_unused_columns=False,  # 防止 Trainer 丢弃你的 labels
    optim="paged_adamw_32bit",  # 使用分页优化器进一步节省显存
    report_to="swanlab",  # 指定汇报给 swanlab
    gradient_checkpointing=True,
    group_by_length=True,
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    data_collator=data_collator,
)

/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [19]:
trainer.train()

/dfs/data/uv-venv/huggingface/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

swanlab: swanlab version 0.7.6 is available!  Upgrade: `pip install -U swanlab`

swanlab: Tracking run with swanlab version 0.7.5

swanlab: Run data will be saved locally in 
/dfs/data/work/hardtry/exps/swanlog/run-20260123_180614-biowomjhdghn41kpp0axy

swanlab: 👋 Hi zhangdw156,welcome to swanlab!

swanlab: Syncing run ox-3 to the cloud

swanlab: 🏠 View project at https://swanlab.cn/@zhangdw156/hardtry

swanlab: 🚀 View run at https://swanlab.cn/@zhangdw156/hardtry/runs/biowomjhdghn41kpp0axy

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/dfs/data/uv-venv/huggingface/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.60 GiB. GPU 0 has a total capacity of 39.39 GiB of which 8.99 GiB is free. Process 1201262 has 30.40 GiB memory in use. Of the allocated memory 24.75 GiB is allocated by PyTorch, and 5.14 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)